# Data Engineer Challenge

### Instalación de liberías necesarias

In [52]:
!pip install memory-profiler
!pip install line_profiler
!pip install emoji
!pip install pandas
!pip install pytest

### Importación y setup

In [53]:
%load_ext memory_profiler
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [5]:
from q1_memory import q1_memory
from q2_memory import q2_memory
from q3_memory import q3_memory
from q1_time import q1_time
from q2_time import q2_time
from q3_time import q3_time
from pandas import DataFrame as visualizador
from etils import ecolab

### Definición de constantes

In [6]:
FILE_PATH = "../data/farmers-protest-tweets-2021-2-4.json"

### ✅ Supuestos realizados

- Se asumió que el campo de fecha está en `"date"` en formato ISO 8601.
- El nombre de usuario se encuentra en `"user.username"`.
- Registros con campos faltantes fueron ignorados.
---

# Pregunta 1 - Tweets por Fecha y Usuario más Activo
> **Objetivo:** Obtener las 10 fechas con más tweets y, para cada una de esas fechas, el usuario más activo (con más publicaciones).

## Enfoque de memoria optimizada

Para minimizar el uso de memoria, implementa las siguientes estrategias:

- Procesa el archivo en modo **streaming** (línea por línea), evitando cargar todo el contenido en memoria.
- Utiliza un **heap limitado** (`min-heap`) para mantener únicamente las **10 fechas con más tweets**, descartando automáticamente las menos relevantes.
- Evita almacenar conteos de todas las fechas posibles, lo que reduce significativamente el uso de memoria frente a otras alternativas.
- Solo conserva información de conteo de usuarios por fecha, necesaria para determinar el usuario más activo en cada una de las 10 fechas seleccionadas.
- Ignora registros incompletos (sin fecha o sin username).
- Convierte las fechas a tipo `datetime.date` solo al final, una vez determinadas las fechas relevantes.


#### Prueba de memoria

In [7]:
%memit q1_memory(FILE_PATH)

peak memory: 111.40 MiB, increment: 6.46 MiB


#### Análisis de memoria

In [49]:
%mprun -f q1_memory q1_memory(FILE_PATH)

Accordion(children=(Output(),), titles=('',))

Filename: C:\Users\and88\OneDrive\Documents\Python\latam_challenge_DE\src\q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     9    497.0 MiB    497.0 MiB           1   def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    10                                             """
    11                                             Procesa un archivo de tweets en formato JSON línea por línea, devolviendo las 10 fechas con más
    12                                             tweets, junto con el nombre de usuario (username) más activo para cada una de esas fechas.
    13                                         
    14                                             Este método está optimizado para un bajo uso de memoria, utilizando un min-heap para mantener
    15                                             únicamente las 10 fechas con mayor cantidad de tweets, en lugar de guardar todos los conteos.
    16                                             Esto

#### Prueba de tiempo

In [8]:
%timeit -n 10 -r 3 q1_memory(FILE_PATH)

3.5 s ± 243 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Resultado

In [9]:
fecha_y_usuarios = q1_memory(FILE_PATH)
visualizador(fecha_y_usuarios, columns=["Fecha", "Usuarios"])

,Fecha,Usuarios
0,2021-02-12,RanbirS00614606
1,2021-02-13,MaanDee08215437
2,2021-02-17,RaaJVinderkaur
3,2021-02-16,jot__b
4,2021-02-14,rebelpacifist
5,2021-02-18,neetuanjle_nitu
6,2021-02-15,jot__b
7,2021-02-20,MangalJ23056160
8,2021-02-23,Surrypuria
9,2021-02-19,Preetm91


## Enfoque de tiempo optimizado

Para minimizar el tiempo de ejecución, esta versión de la función implementa las siguientes estrategias:

- Carga **todo el archivo** de tweets completamente en memoria, reduciendo el tiempo asociado a operaciones de lectura desde disco (I/O).
- Emplea estructuras eficientes (`Counter` y `defaultdict`) para realizar conteos y agrupaciones rápidamente en memoria.
- Evita validaciones o transformaciones costosas dentro del loop principal, procesando los datos de forma directa.
- Realiza las conversiones de strings a `datetime.date` **solamente al final**, una vez que ya se han identificado las 10 fechas con más tweets.
- Ignora registros incompletos (sin fecha o sin username) para evitar errores o procesamiento innecesario.


#### Prueba de memoria

In [10]:
%memit q1_time(FILE_PATH)

peak memory: 522.71 MiB, increment: 412.46 MiB


#### Prueba de tiempo

In [25]:
%timeit -n 10 -r 3 q1_time(FILE_PATH)

3.15 s ± 20.4 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Análisis de tiempo

In [48]:
%lprun -f q1_time q1_time(FILE_PATH)

Accordion(children=(Output(),), titles=('',))

Timer unit: 1e-07 s

Total time: 3.91502 s
File: C:\Users\and88\OneDrive\Documents\Python\latam_challenge_DE\src\q1_time.py
Function: q1_time at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
     9                                               """
    10                                               Procesa un archivo de tweets en formato JSON cargándolo completamente en memoria, devolviendo las
    11                                               10 fechas con más tweets, junto con el nombre de usuario (username) más activo para cada una de esas
    12                                               fechas.
    13                                           
    14                                               Este método está optimizado para minimizar el tiempo de ejecución, cargando el archivo completo en
    15                                        

#### Resultado

In [12]:
fecha_y_usuarios = q1_time(FILE_PATH)
visualizador(fecha_y_usuarios, columns=["Fecha", "Usuarios"])

,Fecha,Usuarios
0,2021-02-12,RanbirS00614606
1,2021-02-13,MaanDee08215437
2,2021-02-17,RaaJVinderkaur
3,2021-02-16,jot__b
4,2021-02-14,rebelpacifist
5,2021-02-18,neetuanjle_nitu
6,2021-02-15,jot__b
7,2021-02-20,MangalJ23056160
8,2021-02-23,Surrypuria
9,2021-02-19,Preetm91


---
# Pregunta 2 - Los top 10 emojis más usados
> **Objetivo:** Obtener los 10 emojis más usados con su respectivo conteo

## Enfoque de memoria optimizada

Para minimizar el uso de memoria, esta versión de la función implementa las siguientes estrategias:

- Procesa el archivo de tweets en modo **streaming** (línea por línea), evitando cargar el archivo completo en memoria.
- Analiza únicamente el campo de texto (`content`) de cada tweet, sin almacenar los textos ni listas completas.
- Utiliza estructuras simples y eficientes (diccionario plano) para contar ocurrencias de emojis.
- Recorre el texto carácter por carácter para identificar emojis de forma directa, sin operaciones costosas adicionales.
- Solo mantiene en memoria el diccionario de conteo acumulado de emojis, que tiende a ser pequeño.
- Ordena y extrae los 10 emojis más frecuentes una vez finalizado el procesamiento completo.

#### Prueba de memoria

In [34]:
%memit q2_memory(FILE_PATH)

peak memory: 483.79 MiB, increment: 0.00 MiB


#### Análisis de memoria

In [45]:
%mprun -f q2_memory q2_memory(FILE_PATH)

Accordion(children=(Output(),), titles=('',))

Filename: C:\Users\and88\OneDrive\Documents\Python\latam_challenge_DE\src\q2_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7    491.2 MiB    491.2 MiB           1   def q2_memory(file_path: str) -> List[Tuple[str, int]]:
     8                                             """
     9                                             Procesa un archivo de tweets en formato JSON línea por línea, devolviendo los 10 emojis más utilizados
    10                                             junto con su cantidad de ocurrencias.
    11                                         
    12                                             Este método está optimizado para un bajo uso de memoria, ya que no carga todo el archivo en
    13                                             memoria, sino que procesa cada tweet individualmente y realiza conteo_de_emojiss incrementales de emojis
    14                                             directamente sobre el texto, sin almacenar datos in

#### Prueba de tiempo

In [14]:
%timeit -n 10 -r 3 q2_memory(FILE_PATH)

5.61 s ± 61.5 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Respuesta

In [15]:
emojis_y_conteo = q2_memory(FILE_PATH)
visualizador(emojis_y_conteo, columns=["Emojis", "Conteos"])

,Emojis,Conteos
0,🙏,7286
1,😂,3072
2,🚜,2972
3,✊,2411
4,🌾,2363
5,🏻,2080
6,❤,1779
7,🤣,1668
8,🏽,1218
9,👇,1108


## Enfoque de tiempo optimizado

Para minimizar el tiempo de ejecución, esta versión de la función implementa las siguientes estrategias:

- Carga **todo el archivo** en memoria utilizando `readlines()`, lo que reduce el tiempo de acceso al disco (I/O).
- Utiliza la estructura `set` construida a partir de `emoji.EMOJI_DATA.keys()` para identificar rápidamente si un carácter es un emoji, con operaciones de búsqueda en tiempo constante.
- Emplea `Counter` de la librería estándar para acumular los conteos de emojis de forma eficiente y rápida.
- Recorre el texto carácter por carácter, sin validaciones adicionales innecesarias dentro del bucle.
- Ignora líneas con errores de decodificación y tweets sin campo `content` para evitar procesamiento extra.
- Extrae los 10 emojis más frecuentes utilizando el método `most_common()` de `Counter`, que es altamente optimizado.


#### Prueba de memoria

In [16]:
%memit q2_time(FILE_PATH)

peak memory: 524.59 MiB, increment: 242.44 MiB


#### Prueba de tiempo

In [17]:
%timeit -n 10 -r 3 q2_time(FILE_PATH)

3.98 s ± 148 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Análisis de tiempo

In [44]:
%lprun -f q2_time q2_time(FILE_PATH)

Accordion(children=(Output(),), titles=('',))

Timer unit: 1e-07 s

Total time: 11.7965 s
File: C:\Users\and88\OneDrive\Documents\Python\latam_challenge_DE\src\q2_time.py
Function: q2_time at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def q2_time(file_path: str) -> List[Tuple[str, int]]:
     8                                               """
     9                                               Procesa un archivo de tweets en formato JSON cargándolo completamente en memoria,
    10                                               y devuelve los 10 emojis más utilizados junto con su cantidad de ocurrencias.
    11                                           
    12                                               Este método está optimizado para velocidad de ejecución, ya que evita la lectura
    13                                               línea por línea (I/O secuencial) y utiliza estructuras eficientes como `set` para
    14                                 

#### Respuesta

In [18]:
emojis_y_conteo = q2_time(FILE_PATH)
visualizador(emojis_y_conteo, columns=["Emojis", "Conteos"])

,Emojis,Conteos
0,🙏,7286
1,😂,3072
2,🚜,2972
3,✊,2411
4,🌾,2363
5,🏻,2080
6,❤,1779
7,🤣,1668
8,🏽,1218
9,👇,1108


---
# Pregunta 3 - Los top 10 usuarios más influyentes
> **Objetivo:** Obtener el top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.

## Enfoque de memoria optimizada

Para minimizar el uso de memoria, esta versión de la función implementa las siguientes estrategias:

- Procesa el archivo **línea por línea** (modo streaming), evitando cargar todo el contenido en memoria.
- Utiliza una función generadora que produce **uno por uno** los nombres de usuario mencionados en cada tweet.
- Solo almacena en memoria los conteos acumulados por usuario utilizando la estructura `Counter`, que es eficiente para este tipo de operaciones.
- Ignora los tweets mal formateados o sin campo `mentionedUsers`, evitando validaciones y estructuras adicionales.
- Devuelve los 10 usuarios más mencionados ordenados por frecuencia, usando el método `most_common()` de `Counter`.

#### Prueba de memoria

In [26]:
%memit q3_memory(FILE_PATH)

peak memory: 460.41 MiB, increment: 0.13 MiB


#### Análisis de memoria

In [46]:
%mprun -f q3_memory q3_memory(FILE_PATH)

Accordion(children=(Output(),), titles=('',))

Filename: C:\Users\and88\OneDrive\Documents\Python\latam_challenge_DE\src\q3_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6    491.5 MiB    491.5 MiB           1   def q3_memory(file_path: str) -> List[Tuple[str, int]]:
     7                                             """
     8                                             Procesa un archivo de tweets en formato JSON línea por línea, devolviendo los 10 usuarios
     9                                             más influyentes, definidos como aquellos que más veces fueron mencionados (@username).
    10                                         
    11                                             Este método está optimizado para un bajo uso de memoria, ya que no carga el archivo completo
    12                                             en memoria, sino que procesa cada tweet individualmente a través de un generador que produce
    13                                             los usernames mencionados 

#### Prueba de tiempo 

In [20]:
%timeit -n 10 -r 3 q3_memory(FILE_PATH)

3.35 s ± 108 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Respuesta

In [21]:
usuarios_mas_influyentes = q3_memory(FILE_PATH)
visualizador(usuarios_mas_influyentes, columns=["Usuario", "Menciones"])

,Usuario,Menciones
0,narendramodi,2265
1,Kisanektamorcha,1840
2,RakeshTikaitBKU,1644
3,PMOIndia,1427
4,RahulGandhi,1146
5,GretaThunberg,1048
6,RaviSinghKA,1019
7,rihanna,986
8,UNHumanRights,962
9,meenaharris,926


### Enfoque de tiempo optimizado

Para minimizar el tiempo de ejecución, esta versión de la función implementa las siguientes estrategias:

- Procesa el archivo línea por línea (modo streaming).
- Utiliza una expresión regular para **filtrar** solo las líneas que contienen al menos una mención válida en el campo `mentionedUsers`, reduciendo significativamente el número de parseos JSON necesarios.
- Solo parsea los tweets que cumplen con ese patrón, lo que mejora el rendimiento en archivos con muchos tweets sin menciones.
- Utiliza la estructura `Counter` para contar eficientemente la cantidad de veces que cada usuario es mencionado.
- Ignora los tweets mal formateados o sin usuarios mencionados, evitando validaciones adicionales o estructuras innecesarias.
- Devuelve los 10 usuarios más mencionados ordenados por frecuencia, utilizando el método `most_common()` de `Counter`.


#### Prueba de memoria

In [22]:
%memit q3_time(FILE_PATH)

peak memory: 427.77 MiB, increment: 0.00 MiB


#### Prueba de tiempo

In [23]:
%timeit -n 10 -r 3 q3_time(FILE_PATH)

2.25 s ± 243 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Análisis de tiempo

In [47]:
%lprun -f q3_time q3_time(FILE_PATH)

Accordion(children=(Output(),), titles=('',))

Timer unit: 1e-07 s

Total time: 2.53946 s
File: C:\Users\and88\OneDrive\Documents\Python\latam_challenge_DE\src\q3_time.py
Function: q3_time at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def q3_time(file_path: str) -> List[Tuple[str, int]]:
     9                                               """
    10                                               Procesa un archivo de tweets en formato JSON línea por línea, devolviendo los 10 usuarios
    11                                               más mencionados (en el campo 'mentionedUsers') a lo largo de todo el historial.
    12                                           
    13                                               Esta versión está optimizada para minimizar el tiempo de ejecución. En lugar de parsear
    14                                               todas las líneas del archivo, utiliza una expresión regular para detectar solo aquellas
    15          

#### Resultado

In [24]:
usuarios_mas_influyentes = q3_time(FILE_PATH)
visualizador(usuarios_mas_influyentes, columns=["Usuario", "Menciones"])

,Usuario,Menciones
0,narendramodi,2265
1,Kisanektamorcha,1840
2,RakeshTikaitBKU,1644
3,PMOIndia,1427
4,RahulGandhi,1146
5,GretaThunberg,1048
6,RaviSinghKA,1019
7,rihanna,986
8,UNHumanRights,962
9,meenaharris,926
